Let's experiment on the [MovieLens 100K Dataset](https://grouplens.org/datasets/movielens/). It includes 100,000 ratings from 1000 users on 1700 movies and was released in 1998. Find the zipfile [here](https://grouplens.org/datasets/movielens/100k/)

In [1]:
import pandas as pd
import zipfile
from pathlib import Path

In [2]:
path = Path('data/ml-100k')
with zipfile.ZipFile(str(path) + '.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
!ls {path}

README       u.genre      u.user       u2.test      u4.test      ua.test
allbut.pl    u.info       u1.base      u3.base      u5.base      ub.base
mku.sh       u.item       u1.test      u3.test      u5.test      ub.test
u.data       u.occupation u2.base      u4.base      ua.base


### Loading the DataFrames

In [6]:
user_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
user_df = pd.read_csv(path/'u.user', sep='|', names=user_cols,
                     encoding='latin-1')
user_df.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [7]:
rating_cols = ['user_id', 'movie_id', 'ratings', 'unix_timestamp']
rating_df = pd.read_csv(path/'u.data', sep='\t', names=rating_cols,
                       encoding='latin-1')
rating_df.head()

,user_id,movie_id,ratings,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
movie_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
# Load only the first 5 columns
movie_df = pd.read_csv(path/'u.item', sep='|', names=movie_cols, 
                       usecols=range(5), encoding='latin-1')
movie_df.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


### Inspecting our DataFrames

In [9]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 5 columns):
movie_id              1682 non-null int64
title                 1682 non-null object
release_date          1681 non-null object
video_release_date    0 non-null float64
imdb_url              1679 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 65.8+ KB


After using the **info( )** method on the *movie_df* dataframe, we find out that the *video_release_date* column isn't populated at all and the columns *release_date* and *imdb_url* are missing few values.

If we are only interesting in getting the datatypes of the different columns, we can use the **dtypes** method.

In [10]:
movie_df.dtypes

movie_id                int64
title                  object
release_date           object
video_release_date    float64
imdb_url               object
dtype: object

To get simple statistics for each numeric column in a DataFrame, we can use the **describe( )** method

In [12]:
user_df.describe()

,user_id,age
count,943.000000,943.000000
mean,472.000000,34.051962
std,272.364951,12.192740
min,1.000000,7.000000
25%,236.500000,25.000000
50%,472.000000,31.000000
75%,707.500000,43.000000
max,943.000000,73.000000


From the result, we can easily spot that the average age of the users part of the study is a shade above 34 years old, with the minimum age being 7 and maximum being 73. 

### Selection

**Column based Selection:** Index the DataFrame with a list of necessary column(s) to make a selection. If you are trying to grab a single column (a Series), you don't need to use a list, just pass the name of the column instead.

In [13]:
user_df.columns

Index(['user_id', 'age', 'sex', 'occupation', 'zip_code'], dtype='object')

In [15]:
cols_needed = ['age', 'occupation']
user_df[cols_needed].head()

,age,occupation
0,24,technician
1,53,other
2,23,writer
3,24,technician
4,33,other


**Row based Selection:**: This can be achieved by using an indice, sliced indexes or boolean masks

In [17]:
mask = user_df['age'] > 60
user_df[mask].head(3)

,user_id,age,sex,occupation,zip_code
105,106,61,M,retired,55125
210,211,66,M,salesman,32605
265,266,62,F,administrator,78756


In [19]:
mask = (user_df.sex == 'F') & (user_df.occupation == 'retired')
user_df[mask].head()

,user_id,age,sex,occupation,zip_code
859,860,70,F,retired,48322


In [20]:
# Index Slice
user_df[200:204]

,user_id,age,sex,occupation,zip_code
200,201,27,M,writer,E2A4H
201,202,41,F,educator,60201
202,203,25,F,student,32301
203,204,52,F,librarian,10960


**A note on Indexing:** As you may have noticed, Pandas adds a default numeric indice to each row of the dataframe, acting as an unique key. If we would like to change this to an existing column, we can just use the **set_index( )** method.

In [21]:
# set_index returns a new dataframe. To modify the existing dataframe, use inplace=True param
clean_user_df = user_df.set_index('user_id')
clean_user_df.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


We can use the convenient **iloc** method to grab the rows based on this new primary key.

In [22]:
print(clean_user_df.iloc[100])
print('\n')
print(clean_user_df.iloc[[1, 100, 200]])

age                15
sex                 M
occupation    student
zip_code        05146
Name: 101, dtype: object


         age sex occupation zip_code
user_id                             
2         53   F      other    94043
101       15   M    student    05146
201       27   M     writer    E2A4H


As you can see, **iloc** is giving you the rows at particular positions in the index. If instead, you want to get the rows based on the label/value of the index, use the **loc** method. See if you can spot the difference.

In [23]:
print(clean_user_df.loc[100])
print('\n')
print(clean_user_df.loc[[1, 100, 200]])

age                  36
sex                   M
occupation    executive
zip_code          90254
Name: 100, dtype: object


         age sex  occupation zip_code
user_id                              
1         24   M  technician    85711
100       36   M   executive    90254
200       40   M  programmer    93402
